## Part 1

The first part consists of loading the required libraries,creating the S3 bucket and uploading the csv file. We can create the s3 bucket directly from aws console also.

In [65]:
###Load modules
#!pip install webdriver-manager
import awscli
import boto3
import selenium
import pandas as pd
import time
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from botocore.exceptions import ClientError

####SCRAPE THE WEBSITE######
###call the webdriver
s = Service(ChromeDriverManager().install())
browser = webdriver.Chrome(service=s)

#enter the url path that needs to be accessed by webdriver
browser.get('https://www.charitiesnys.com/RegistrySearch/search_charities.jsp')

#identify xpath of location to select element
inputElement = browser.find_element(
    By.XPATH, "/html/body/div/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[2]/td[2]/input[1]")
inputElement.send_keys('0')
inputElement1 = browser.find_element(
    By.XPATH, "/html/body/div/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[10]/td/input[1]").click()
sleep(4)  # allow for the page to load by adding a sleep element
#identify the table to scrape
table = browser.find_element(By.CSS_SELECTOR, 'table.Bordered')
sleep(1)


In [66]:
#####CREATE DATE FRAME#####
#create empty dataframe
df = []

#loop through dataframe to export table
for row in table.find_elements(By.CSS_SELECTOR, 'tr'):
    cols = df.append(
        [cell.text for cell in row.find_elements(By.CSS_SELECTOR, 'td')])


#update dataframe with header
df = pd.DataFrame(df, columns=[
                  "Organization Name", "NY Reg #", "EIN", "Registrant Type", "City", "State"])

# let's have a look at the data before creating the CSV file and loading it into s3
display(df)


,Organization Name,NY Reg #,EIN,Registrant Type,City,State
1,"""Forever Captain Poodaman"" The Ahmad Butler Fo...",48-07-16,843800926,NFP,PHILADELPHIA,PA
2,"""R"" S.U.C.C.E.S.S. Foundation Inc.",49-06-59,874012670,NFP,ROCHESTER,NY
3,"""Studio 5404"" Inc.",44-39-58,463180470,NFP,MASSAPAQUA,NY
4,"""THEY ARE HAITIAN"" FUND, INC.",20-63-46,300170128,NFP,HUDSON,NY
5,"""Y"" Dive, Inc.",48-45-01,854252095,NFP,SAINT ALBANS,NY
6,(ASMA) American Syrian Multicultural Associati...,42-84-63,273130182,NFP,BROOKLYN,NY
7,#FeedHamburg,48-37-35,854150318,NFP,HAMBURG,NY
8,#HicksStrong Inc.,48-10-48,842612081,NFP,CLIFTON PARK,NY
9,#WalkAway Foundation,47-15-80,832820906,NFP,CARLSBAD,CA
10,04/11 10:17 PM test,47-13-95,206256427,NFP,ALBANY,NY


In [63]:


# # Set the name of the bucket to create
bucket_name = 'm10-assignment'

aws_access_key = input("enter your access key \n")
aws_secret_key = input("enter your secret key \n")

aws_s3_client = boto3.client('s3', aws_access_key_id=aws_access_key,
                             aws_secret_access_key=aws_secret_key)

# Below commented code has the access key to my AWS account.
response = aws_s3_client.list_buckets()
bucket_exist = False

for bucket in response['Buckets']:
    if bucket['Name'] == bucket_name:
        bucket_exist = True
        break

if bucket_exist:
    print("The bucket exists")
else:
    print("The bucket does not exist")

# Create the bucket if it doesn't exist
if not bucket_exist:
    try:
        aws_s3_client.create_bucket(Bucket=bucket_name)
        print(f"{bucket_name} bucket has been created on AWS S3")
    except ClientError as e:
        print(e)
        print(f"{bucket_name} cannot be created on S3")


The bucket exists


In [64]:

pathname = 's3://m10-assignment/'
filename = 'charities_bureau_scrape_group6'  # name of your group
datetime = time.strftime("%Y-%m-%d-%H-%M-%S")  # timestamp
# name of the filepath and csv file
filenames3 = f"{filename}-{datetime}.csv"

def upload_s3(df, i):
    global aws_s3_client, bucket_name
    csv_buffer = StringIO()
    df.to_csv(csv_buffer, header=True, line_terminator='\n')
    csv_buffer.seek(0)
    aws_s3_client.put_object(
        Bucket=bucket_name, Body=csv_buffer.getvalue(), Key=i, ACL='public-read')
print("Uploading Data")
upload_s3(df, filenames3)
print("Data uploaded successfully ")

#print success message
print(f"Successfully uploaded file to location:{pathname}{filenames3}")


Uploading Data
Data uploaded successfully 
Successfully uploaded file to location:s3://m10-assignment/charities_bureau_scrape_group6-2023-04-12-19-37-58.csv


In [41]:
#link to csv file
https://m10-assignment.s3.amazonaws.com/charities_bureau_scrape_group6-2023-04-12-18-29-00.csv